In [57]:
# imports
import requests
import json
import numpy as np
import pandas as pd

base_url = 'https://api-web.nhle.com/v1/'
# API TEST
url = 'https://api-web.nhle.com/v1/roster/VAN/20242025'
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
else:
    print(f'Error: {response.status_code}')

print(data)

{'forwards': [{'id': 8483395, 'headshot': 'https://assets.nhle.com/mugs/nhl/20242025/VAN/8483395.png', 'firstName': {'default': 'Arshdeep'}, 'lastName': {'default': 'Bains'}, 'sweaterNumber': 13, 'positionCode': 'L', 'shootsCatches': 'L', 'heightInInches': 72, 'weightInPounds': 184, 'heightInCentimeters': 183, 'weightInKilograms': 83, 'birthDate': '2001-01-09', 'birthCity': {'default': 'Surrey'}, 'birthCountry': 'CAN', 'birthStateProvince': {'default': 'BC'}}, {'id': 8476927, 'headshot': 'https://assets.nhle.com/mugs/nhl/20242025/VAN/8476927.png', 'firstName': {'default': 'Teddy', 'cs': 'Theodors', 'sk': 'Theodors'}, 'lastName': {'default': 'Blueger', 'cs': 'Blugers', 'sk': 'Blugers'}, 'sweaterNumber': 53, 'positionCode': 'C', 'shootsCatches': 'L', 'heightInInches': 72, 'weightInPounds': 185, 'heightInCentimeters': 183, 'weightInKilograms': 84, 'birthDate': '1994-08-15', 'birthCity': {'default': 'Riga', 'fi': 'Riika'}, 'birthCountry': 'LVA'}, {'id': 8478444, 'headshot': 'https://assets

## Get all team data

In [49]:
url = "https://api.nhle.com/stats/rest/en/team/summary?cayenneExp=seasonId=20242025"

response = requests.get(url)

team_data = response.json()
team_ids = []
for team in team_data['data']:
    team_ids.append(team['teamId'])

print(team_ids)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 28, 29, 30, 52, 54, 55, 59]


In [50]:
import requests

# URL to fetch data from current teams
current_teams_url = 'https://api.nhle.com/stats/rest/en/team/summary?cayenneExp=seasonId=20242025'

# Send GET request to NHL API
response = requests.get(current_teams_url)
current_team_data = response.json()

# Retrieve team IDs and abbreviations
team_ids = [team['teamId'] for team in current_team_data['data']]
#print("Team IDs:", team_ids)

teams_url = 'https://api.nhle.com/stats/rest/en/team'
team_data = requests.get(teams_url).json()
#print(team_data)
team_dict = {}


for data in team_data['data']:
    for id in team_ids:
        if data['id'] == id:
            team_dict[id] = data['triCode']

team_dict = dict(sorted(team_dict.items()))

## Get all players for each team
endpoint: /v1/roster/{team}/current

In [51]:
player_ids = []
team_count = 0
for item in team_dict:
    team_count += 1
    print(team_count)
    team_abbrv = team_dict[item]
    roster_url_endpoint = f'roster/{team_abbrv}/current'
    final_url = base_url + roster_url_endpoint

    print(final_url)
    response = requests.get(final_url)
    roster_data = response.json()
    
    for forward in roster_data['forwards']:
        player_ids.append(forward['id'])

    for defense in roster_data['defensemen']:
        player_ids.append(defense['id'])

    for goalie in roster_data['goalies']:
        player_ids.append(goalie['id'])

    print('current len: ', len(player_ids))

print(len(player_ids))

1
https://api-web.nhle.com/v1/roster/NJD/current
current len:  26
2
https://api-web.nhle.com/v1/roster/NYI/current
current len:  52
3
https://api-web.nhle.com/v1/roster/NYR/current
current len:  75
4
https://api-web.nhle.com/v1/roster/PHI/current
current len:  102
5
https://api-web.nhle.com/v1/roster/PIT/current
current len:  128
6
https://api-web.nhle.com/v1/roster/BOS/current
current len:  151
7
https://api-web.nhle.com/v1/roster/BUF/current
current len:  176
8
https://api-web.nhle.com/v1/roster/MTL/current
current len:  198
9
https://api-web.nhle.com/v1/roster/OTT/current
current len:  220
10
https://api-web.nhle.com/v1/roster/TOR/current
current len:  249
11
https://api-web.nhle.com/v1/roster/CAR/current
current len:  271
12
https://api-web.nhle.com/v1/roster/FLA/current
current len:  293
13
https://api-web.nhle.com/v1/roster/TBL/current
current len:  315
14
https://api-web.nhle.com/v1/roster/WSH/current
current len:  343
15
https://api-web.nhle.com/v1/roster/CHI/current
current le

populating a player csv table

In [61]:
for player_id in player_ids:
    try:
        player_endpoint = f'player/{player_id}/landing'
        final_url = base_url + player_endpoint
        response = requests.get(final_url)

        if response.status_code != 200:
            print(f"Skipping player ID {player_id}: Status code {response.status_code}")
            continue

        player_data = response.json()

        # Skip if required fields are missing
        if not all(key in player_data for key in ["playerId", "firstName", "lastName", "careerTotals"]):
            print(f"Missing data for player ID {player_id}")
            continue

        player_info = {
            "Player ID": player_data["playerId"],
            "First Name": player_data["firstName"].get("default", "N/A"),
            "Last Name": player_data["lastName"].get("default", "N/A"),
            "Team Abbreviation": player_data.get("currentTeamAbbrev", "N/A"),
            "Sweater Number": player_data.get("sweaterNumber", "N/A"),
            "Position": player_data.get("position", "N/A"),
            "Goals": player_data["careerTotals"]["regularSeason"].get("goals", 0),
            "Assists": player_data["careerTotals"]["regularSeason"].get("assists", 0),
            "Points": player_data["careerTotals"]["regularSeason"].get("points", 0),
            "Games Played": player_data["careerTotals"]["regularSeason"].get("gamesPlayed", 0),
            "Plus/Minus": player_data["careerTotals"]["regularSeason"].get("plusMinus", 0),
        }

        player_table = pd.concat([player_table, pd.DataFrame([player_info])], ignore_index=True)

    except KeyError as e:
        print(f"KeyError for player ID {player_id}: {e}")
        continue
    except Exception as e:
        print(f"Unexpected error for player ID {player_id}: {e}")
        continue


Missing data for player ID 8482482
Missing data for player ID 8484136
Missing data for player ID 8481707


In [62]:
player_table.head()

,Player ID,First Name,Last Name,Team Abbreviation,Sweater Number,Position,Goals,Assists,Points,Games Played,Plus/Minus
0,8479414,Nathan,Bastian,NJD,14,R,31,32,63,229,-14
1,8480032,Shane,Bowers,NJD,15,C,0,0,0,10,1
2,8479407,Jesper,Bratt,NJD,63,L,139,247,386,493,-14
3,8481032,Paul,Cotter,NJD,47,C,29,28,57,160,-17
4,8475413,Justin,Dowling,NJD,37,C,7,13,20,110,-19
